In [ ]:
import os
import pytesseract as pt
from PIL import Image as im
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.utils import np_utils
from sklearn.utils import shuffle


In [68]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D ,Conv3D
from keras.layers import Input, Dense, Activation ,Flatten
from keras.layers import Reshape, Lambda 
from keras.layers.merge import add, concatenate
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image as ki
from keras.layers.convolutional_recurrent import ConvLSTM2D

In [14]:
def get_processed_images(invoice=True):
    if invoice :
        image_dir =  "purchase_order\\" 
    else:
        image_dir='flower_images\\' 
    size = (224,224)
    normalized_image = []
    imgV = []
    for pic in os.listdir(image_dir):
        img = im.open(image_dir+pic)
        img =img.resize(size,im.ANTIALIAS) 
        img_array = ki.img_to_array(img)
       
        v_min = img_array.min(axis=(0, 1), keepdims=True)
        v_max = img_array.max(axis=(0, 1), keepdims=True)
        img_array = (img_array - v_min)/(v_max - v_min)
        normalized_image.append(img_array)
        filterImage = filter(lambda x : x.shape[2]==3 ,normalized_image)
        imageData = np.array(list(filterImage))
    return imageData

    


In [15]:
from sklearn.model_selection import train_test_split
invoiceData = get_processed_images(True)
nonInvoiceData = get_processed_images(False)

image_data = np.concatenate([invoiceData,nonInvoiceData])

C:\Users\PwC Hyd\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [16]:
Y=np.zeros((len(nonInvoiceData),)) # ALL THE ZEROES ARE NON INVOICE/ FLOWER DATA 
labels = np.ones((len(invoiceData),)) #ALL THE ONES ARE INVOICE DATA

Y= np.concatenate([labels,Y])
Y = np_utils.to_categorical(Y, 2)

x,y = shuffle(image_data,Y, random_state=2)

X_train,X_test,y_train,y_test = train_test_split(x,y, train_size=0.67 ,test_size=0.33)


In [20]:
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')


In [72]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer) 
x= Dense(128,activation='relu',name='fc1')(x)
x= Dense(128,activation='relu',name='fc2')(x)
out = Dense(2, activation='softmax', name='output')(x)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False


custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

<tf.Tensor 'output_5/Softmax:0' shape=(?, 2) dtype=float32>

In [73]:
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))


Train on 109 samples, validate on 54 samples
Epoch 1/12
109/109 [==============================] - 45s 409ms/step - loss: 0.2438 - acc: 0.8807 - val_loss: 8.3529 - val_acc: 0.4815
Epoch 2/12
109/109 [==============================] - 47s 434ms/step - loss: 6.3019 - acc: 0.6055 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 3/12
109/109 [==============================] - 50s 459ms/step - loss: 1.1569e-04 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 4/12
109/109 [==============================] - 49s 451ms/step - loss: 4.0437e-06 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 5/12
109/109 [==============================] - 49s 447ms/step - loss: 3.2490e-06 - acc: 1.0000 - val_loss: 9.9890e-04 - val_acc: 1.0000
Epoch 6/12
109/109 [==============================] - 49s 447ms/step - loss: 2.6331e-06 - acc: 1.0000 - val_loss: 9.1537e-04 - val_acc: 1.0000
Epoch 7/12
109/109 [==============================] - 47s 434ms/step - loss: 1.9545e-06 - acc: 1.0000 - val_loss: 7.7018e

array([0., 1.], dtype=float32)

In [114]:
size=(224,224)
img =  im.open('download.jpg')
img = img.resize(size,im.ANTIALIAS) 
image_to_predict  = ki.img_to_array(img)
image_to_predict = np.expand_dims(image_to_predict,axis=0)

In [117]:
hist.model.save_weights('InvoiceClassifier.h5')